## 0. Library Import

In [181]:
import pandas as pd 
import seaborn as sns
import numpy as np 
import requests
import inspect
import re

import OpenDartReader
import dart_fss as dart
from pykrx import stock
from pykrx import bond
from scipy.stats import zscore

In [182]:
# 객체 생성
api_key = "af983cf5135fd27b1b42edb107d1854207fd26c6"
# DATR에서 API키 설정 후 요청 인증
dart.set_api_key(api_key=api_key)
# DART 기업 목록
corp_list = dart.get_corp_list()

## 1. 데이터 수집

### 1-1. 종목코드, 종목명 리스트

In [179]:
kospi200 = pd.read_csv('./data/kospi200_0826.csv',encoding='euc-kr')
kospi200['종목코드'] = kospi200['종목코드'].astype('str')

def add_zeros(code):
    if len(code) == 2:
        return '0000' + code
    elif len(code) == 3:
        return '000' + code
    elif len(code) == 4:
        return '00' + code
    elif len(code) == 5:
        return '0' + code
    else:
        return code

    
# '이름' 열에 사용자 정의 함수 적용
kospi200['종목코드_process'] = kospi200['종목코드'].apply(add_zeros)
kospi200_code_name = kospi200[['종목코드_process','종목명']]
kospi200_code_name = kospi200_code_name.rename(columns = {'종목코드_process':'종목코드'})
kospi200_code_name

,종목코드,종목명
0,005930,삼성전자
1,373220,LG에너지솔루션
2,000660,SK하이닉스
3,207940,삼성바이오로직스
4,005490,POSCO홀딩스
...,...,...
195,020000,한섬
196,381970,케이카
197,057050,현대홈쇼핑
198,284740,쿠쿠홈시스


### 1-2. Dart Data 전처리 함수

In [52]:
cols = ['Total Asset','Working Asset','Total Equity','Total Liabilities','Retained Earnings','Revenue','Gross Profit','Operating Income','Net Income','EBIT','Operating Cashflow']

In [230]:
def preprocess_dart1(bs,cis,Is,cf):
    # 행렬 변환
    bs = bs.transpose() 
    cis = cis.transpose()
    Is = Is.transpose()
    cf = cf.transpose()

    # header지정 및 불필요한 칼럼 제거
    new_header = bs.iloc[0]
    bs = bs[1:]
    bs.columns = new_header
    bs.drop(bs.index[0:2],inplace= True)
    
    new_header = cis.iloc[0]
    cis = cis[1:]
    cis.columns = new_header
    cis.drop(cis.index[0:2],inplace= True)

    new_header = Is.iloc[0]
    Is = Is[1:]
    Is.columns = new_header
    Is.drop(Is.index[0:2],inplace= True)    
    
    new_header = cf.iloc[0]
    cf = cf[1:]
    cf.columns = new_header
    cf.drop(cf.index[0:2],inplace= True)

    return bs, cis, Is, cf 

def preprocess_dart2(bs,cis,Is,cf):
    # 계산을 위한 index reset, 기존 연도 index는 내림차순
    
    new_index = [idx[0].split('\t')[0][:4] for idx in bs.index]
    bs.index = new_index
    new_index = [idx[0].split('\t')[0][:4] for idx in cis.index]
    cis.index = new_index
    new_index = [idx[0].split('\t')[0][:4] for idx in Is.index]
    Is.index = new_index
    new_index = [idx[0].split('\t')[0][:4] for idx in cf.index]
    cf.index = new_index

    # 계산을 위해 balance sheet, income statement, cashflow merge
    merged_df = pd.concat([bs,cis,Is,cf], axis=1)
    return merged_df

In [184]:
# 데이터 프레임의 이름을 저장하기 위한 메소드 
def dataframe_name(df):
    frame = inspect.currentframe().f_back
    variable_name = [k for k, v in frame.f_globals.items() if v is df][0]
    return variable_name

# 인덱스의 이름을 해당 기업의 이름으로 바꾸기 위한 메소드 
def change_index(df,df2):
    name = dataframe_name(df2)
    df = df.rename(index = {0: name})
    return df 

#### Ex. 삼성전자

In [223]:
# bs : 재무상태표, Is : 손익계산서(포괄손익계산서 대신 사용), cf : 현금흐름표
ticker = kospi200_code_name['종목코드'][0]
df = dart.fs.extract(ticker, '20160101', end_de='20231231', fs_tp=('bs','cis','is','cf'), separate=False, report_tp='annual', lang='ko', separator=True, dataset='xbrl', cumulative=False, progressbar=True, skip_error=True,last_report_only=True)
df.to_dict()

Annual reports:   0%|          | 0/8 [00:00<?, ?report/s]

{'corp_code': '005930',
 'bgn_de': '20160101',
 'end_de': '20231231',
 'separate': False,
 'report_tp': 'annual',
 'lang': 'ko',
 'separator': True,
 'financial statement': [{'title': '[D210000] Statement of financial position, current/non-current - Consolidated financial statements (Unit: KRW)'},
  {'title': '[D310000] Income statement, by function of expense - Consolidated financial statements (Unit: KRW)'},
  {'title': '[D410000] Statement of comprehensive income - Consolidated financial statements (Unit: KRW)'},
  {'title': '[D520000] Statement of cash flows, indirect method - Consolidated financial statements (Unit: KRW)'}]}

In [231]:
# 전처리
# (bgn_df - 3년) ~ (end_df - 2년)
bs = df.show('bs',show_class = False,show_depth = 0,show_concept = False)
cis = df.show('cis',show_class = False,show_depth = 0,show_concept=False)
Is = df.show('is',show_class = False,show_depth = 0,show_concept=False)
cf = df.show('cf',show_class = False,show_depth = 0,show_concept=False)
bs,cis,Is,cf = preprocess_dart1(bs,cis,Is,cf)
bs

,"([D210000] Statement of financial position, current/non-current - Consolidated financial statements (Unit: KRW), label_ko)",유동자산,현금및현금성자산,단기금융상품,단기상각후원가금융자산,단기당기손익-공정가치금융자산,매출채권,미수금,선급비용,재고자산,기타유동자산,...,자본금,우선주자본금,보통주자본금,주식발행초과금,이익잉여금(결손금),기타자본항목,매각예정분류기타자본항목,비지배지분,자본총계,부채와자본총계
20211231,"(연결재무제표,)","218,163,185,000,000.0","39,031,415,000,000.0","81,708,986,000,000.0","3,369,034,000,000.0","40,757,000,000.0","40,713,415,000,000.0","4,497,257,000,000.0","2,336,252,000,000.0","41,384,404,000,000.0","5,081,665,000,000.0",...,"897,514,000,000.0","119,467,000,000.0","778,047,000,000.0","4,403,893,000,000.0","293,064,763,000,000.0","-2,128,473,000,000.0",0.0,"8,662,234,000,000.0","304,899,931,000,000.0","426,621,158,000,000.0"
20201231,"(연결재무제표,)","198,215,579,000,000.0","29,382,578,000,000.0","92,441,703,000,000.0","2,757,111,000,000.0","71,451,000,000.0","30,965,058,000,000.0","3,604,539,000,000.0","2,266,100,000,000.0","32,043,145,000,000.0","3,754,462,000,000.0",...,"897,514,000,000.0","119,467,000,000.0","778,047,000,000.0","4,403,893,000,000.0","271,068,211,000,000.0","-8,687,155,000,000.0","-12,132,000,000.0","8,277,685,000,000.0","275,948,016,000,000.0","378,235,718,000,000.0"
20191231,"(연결재무제표,)","181,385,260,000,000.0","26,885,999,000,000.0","76,252,052,000,000.0","3,914,216,000,000.0","1,727,436,000,000.0","35,131,343,000,000.0","4,179,120,000,000.0","2,406,220,000,000.0","26,766,464,000,000.0","4,122,410,000,000.0",...,"897,514,000,000.0","119,467,000,000.0","778,047,000,000.0","4,403,893,000,000.0","254,582,894,000,000.0","-4,968,829,000,000.0",0.0,"7,964,949,000,000.0","262,880,421,000,000.0","352,564,497,000,000.0"
20181231,"(연결재무제표,)","174,697,424,000,000.0","30,340,505,000,000.0","65,893,797,000,000.0","2,703,693,000,000.0","2,001,948,000,000.0","33,867,733,000,000.0","3,080,733,000,000.0","4,136,167,000,000.0","28,984,704,000,000.0","3,688,144,000,000.0",...,"897,514,000,000.0","119,467,000,000.0","778,047,000,000.0","4,403,893,000,000.0","242,698,956,000,000.0","-7,931,370,000,000.0",0.0,"7,684,184,000,000.0","247,753,177,000,000.0","339,357,244,000,000.0"
20171231,"(연결재무제표,)","146,982,464,000,000.0","30,545,130,000,000.0","49,447,696,000,000.0",NaN,NaN,"27,695,995,000,000.0","4,108,961,000,000.0","3,835,219,000,000.0","24,983,355,000,000.0","1,421,060,000,000.0",...,"897,514,000,000.0","119,467,000,000.0","778,047,000,000.0","4,403,893,000,000.0","215,811,200,000,000.0","-13,899,191,000,000.0",NaN,"7,278,012,000,000.0","214,491,428,000,000.0","301,752,090,000,000.0"
20161231,"(연결재무제표,)","141,429,704,000,000.0","32,111,442,000,000.0","52,432,411,000,000.0",NaN,NaN,"24,279,211,000,000.0","3,521,197,000,000.0","3,502,083,000,000.0","18,353,503,000,000.0","1,315,653,000,000.0",...,"897,514,000,000.0","119,467,000,000.0","778,047,000,000.0","4,403,893,000,000.0","193,086,317,000,000.0","-11,934,586,000,000.0","-28,810,000,000.0","6,538,705,000,000.0","192,963,033,000,000.0","262,174,324,000,000.0"
20151231,"(연결재무제표,)","124,814,725,000,000.0","22,636,744,000,000.0","44,228,800,000,000.0",NaN,NaN,"25,168,026,000,000.0","3,352,663,000,000.0","3,170,632,000,000.0","18,811,794,000,000.0","1,035,460,000,000.0",...,"897,514,000,000.0","119,467,000,000.0","778,047,000,000.0","4,403,893,000,000.0","185,132,014,000,000.0","-17,580,451,000,000.0","23,797,000,000.0","6,183,038,000,000.0","179,059,805,000,000.0","242,179,521,000,000.0"
20141231,"(연결재무제표,)","115,146,026,000,000.0","16,840,766,000,000.0","41,689,776,000,000.0",NaN,NaN,"24,694,610,000,000.0","3,539,875,000,000.0","3,346,593,000,000.0","17,317,504,000,000.0","1,795,143,000,000.0",...,"897,514,000,000.0","119,467,000,000.0","778,047,000,000.0","4,403,893,000,000.0","169,529,604,000,000.0","-12,729,387,000,000.0","80,101,000,000.0","5,906,463,000,000.0","168,088,188,000,000.0","230,422,958,000,000.0"
20131231,"(연결재무제표,)","110,760,271,000,000.0","16,284,780,000,000.0","36,722,702,000,000.0",NaN,NaN,"24,988,532,000,000.0","2,887,402,000,000.0","2,472,950,000,000.0","19,1

In [232]:
# Quality 계산을 위해 필요한 column
SAMSUNG = preprocess_dart2(bs,cis,Is,cf)
SAMSUNG

,유동자산,현금및현금성자산,단기금융상품,단기상각후원가금융자산,단기당기손익-공정가치금융자산,매출채권,미수금,선급비용,재고자산,기타유동자산,...,단기차입금의 순증가(감소),장기차입금의 차입,사채 및 장기차입금의 상환,배당금의 지급,비지배지분의 증감,매각예정분류,외화환산으로 인한 현금의 변동,현금및현금성자산의 순증감,기초의 현금및현금성자산,기말의 현금및현금성자산
2021,"218,163,185,000,000.0","39,031,415,000,000.0","81,708,986,000,000.0","3,369,034,000,000.0","40,757,000,000.0","40,713,415,000,000.0","4,497,257,000,000.0","2,336,252,000,000.0","41,384,404,000,000.0","5,081,665,000,000.0",...,"-2,616,943,000,000.0","58,279,000,000.0","894,749,000,000.0","20,510,350,000,000.0","-27,270,000,000.0","139,000,000.0","1,582,046,000,000.0","9,648,837,000,000.0","29,382,578,000,000.0","39,031,415,000,000.0"
2020,"198,215,579,000,000.0","29,382,578,000,000.0","92,441,703,000,000.0","2,757,111,000,000.0","71,451,000,000.0","30,965,058,000,000.0","3,604,539,000,000.0","2,266,100,000,000.0","32,043,145,000,000.0","3,754,462,000,000.0",...,"2,191,186,000,000.0","14,495,000,000.0","864,947,000,000.0","9,676,760,000,000.0","8,187,000,000.0","-139,000,000.0","-833,861,000,000.0","2,496,579,000,000.0","26,885,999,000,000.0","29,382,578,000,000.0"
2019,"181,385,260,000,000.0","26,885,999,000,000.0","76,252,052,000,000.0","3,914,216,000,000.0","1,727,436,000,000.0","35,131,343,000,000.0","4,179,120,000,000.0","2,406,220,000,000.0","26,766,464,000,000.0","4,122,410,000,000.0",...,"865,792,000,000.0",0.0,"709,400,000,000.0","9,639,202,000,000.0","-1,700,000,000.0",0.0,"595,260,000,000.0","-3,454,506,000,000.0","30,340,505,000,000.0","26,885,999,000,000.0"
2018,"174,697,424,000,000.0","30,340,505,000,000.0","65,893,797,000,000.0","2,703,693,000,000.0","2,001,948,000,000.0","33,867,733,000,000.0","3,080,733,000,000.0","4,136,167,000,000.0","28,984,704,000,000.0","3,688,144,000,000.0",...,"-2,046,470,000,000.0","3,580,000,000.0","1,986,597,000,000.0","10,193,695,000,000.0","8,071,000,000.0",0.0,"94,187,000,000.0","-204,625,000,000.0","30,545,130,000,000.0","30,340,505,000,000.0"
2017,"146,982,464,000,000.0","30,545,130,000,000.0","49,447,696,000,000.0",NaN,NaN,"27,695,995,000,000.0","4,108,961,000,000.0","3,835,219,000,000.0","24,983,355,000,000.0","1,421,060,000,000.0",...,"2,730,676,000,000.0","998,311,000,000.0","1,140,803,000,000.0","6,804,297,000,000.0","5,670,000,000.0",NaN,"-1,782,270,000,000.0","-1,566,312,000,000.0","32,111,442,000,000.0","30,545,130,000,000.0"
2016,"141,429,704,000,000.0","32,111,442,000,000.0","52,432,411,000,000.0",NaN,NaN,"24,279,211,000,000.0","3,521,197,000,000.0","3,502,083,000,000.0","18,353,503,000,000.0","1,315,653,000,000.0",...,"1,351,037,000,000.0","1,041,743,000,000.0","252,846,000,000.0","3,114,742,000,000.0","13,232,000,000.0",NaN,"417,243,000,000.0","9,474,698,000,000.0","22,636,744,000,000.0","32,111,442,000,000.0"
2015,"124,814,725,000,000.0","22,636,744,000,000.0","44,228,800,000,000.0",NaN,NaN,"25,168,026,000,000.0","3,352,663,000,000.0","3,170,632,000,000.0","18,811,794,000,000.0","1,035,460,000,000.0",...,"3,202,416,000,000.0","192,474,000,000.0","1,801,465,000,000.0","3,129,544,000,000.0","-25,312,000,000.0",NaN,"-524,487,000,000.0","5,795,978,000,000.0","16,840,766,000,000.0","22,636,744,000,000.0"
2014,"115,146,026,000,000.0","16,840,766,000,000.0","41,689,776,000,000.0",NaN,NaN,"24,694,610,000,000.0","3,539,875,000,000.0","3,346,593,000,000.0","17,317,504,000,000.0","1,795,143,000,000.0",...,"1,833,419,000,000.0","1,740,573,000,000.0","3,299,595,000,000.0","2,233,905,000,000.0","139,000,000.0",NaN,"-555,886,000,000.0","555,986,000,000.0","16,284,780,000,000.0","16,840,766,000,000.0"
2013,"110,760,271,000,000.0","16,284,780,000,000.0","36,722,702,000,000.0",NaN,NaN,"24,988,532,000,000.0","2,887,402,000,000.0","2,472,950,000,000.0","19,134,868,000,000.0","2,135,589,000,000.0",...,"-1,861,536,000,000.0","26,672,000,000.0","1,368,436,000,000.0","1,249,672,000,000.0","281,551,000,000.0",NaN,"-330,070,000,000.0","-2,506,680,000,000.0","18,791,460,000,000.0","16,284,780,000,000.0"


In [234]:
def clean_company_name(company_name):
    # Remove invalid characters using regular expressions
    clean_name = re.sub(r'[\\/:"*?<>|]+', '', company_name)
    return clean_name

for ticker, company_name in zip(kospi200_code_name['종목코드'], kospi200_code_name['종목명']):
    try:
        # Clean the company name
        cleaned_company_name = clean_company_name(company_name)

        # bs: 재무상태표, Is: 손익계산서(포괄손익계산서 대신 사용), cf: 현금흐름표
        df = dart.fs.extract(ticker, '20160101', end_de='20231231', fs_tp=('bs','cis','is','cf'), separate=False, report_tp='annual', lang='ko', separator=True, dataset='xbrl', cumulative=False, progressbar=True, skip_error=True,last_report_only=True)
        df.to_dict()

        bs = df.show('bs', show_class=False, show_depth=0, show_concept=False)
        cis = df.show('cis', show_class=False, show_depth=0, show_concept=False)
        Is = df.show('is', show_class=False, show_depth=0, show_concept=False)
        cf = df.show('cf', show_class=False, show_depth=0, show_concept=False)
        bs, cis, Is, cf = preprocess_dart1(bs, cis, Is, cf)

        # Quality 계산을 위해 필요한 column
        stock_balance = preprocess_dart2(bs, cis, Is, cf)

        # 파일로 저장
        stock_balance.to_csv(f'./stock_balance_data/{cleaned_company_name}_balance.csv', encoding='euc-kr')

    except Exception as e:
        print(f"Error for ticker {ticker}: {str(e)}")
        pass

Annual reports:   0%|          | 0/2 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/2 [00:00<?, ?report/s]